# Recreating the correct merged dataset

In [1]:
import pandas as pd
import glob
import os

In [2]:
path = "data/raw/**/*.csv"  
all_files = glob.glob(path, recursive=True)

len(all_files)

36

In [3]:
# Combining the files 

df_list = []

for file in all_files:
    temp = pd.read_csv(file, low_memory=False)
    df_list.append(temp)

trips = pd.concat(df_list, ignore_index=True)

trips.shape

(29838806, 13)

In [4]:
# Filter

trips["started_at"] = pd.to_datetime(trips["started_at"], errors="coerce")

trips_2022 = trips[
    (trips["started_at"] >= "2022-01-01") &
    (trips["started_at"] < "2023-01-01")
]

In [5]:
trips_2022["started_at"].min(), trips_2022["started_at"].max()

(Timestamp('2022-01-01 00:00:13.532000'),
 Timestamp('2022-12-31 23:58:19.206000'))

# Creating the merge key

In [6]:
trips_2022["date"] = trips_2022["started_at"].dt.date

In [8]:
weather = pd.read_csv("data/processed/noaa_lga_daily_2022.csv")
weather.columns

Index(['date', 'PRCP', 'TMAX', 'TMIN'], dtype='str')

In [9]:
trips_2022.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'date'],
      dtype='str')

In [10]:
citibike_weather = trips_2022.merge(weather, on="date", how="left")

In [11]:
citibike_weather.shape

(29838166, 17)

In [12]:
citibike_weather[["started_at","PRCP","TMAX"]].head()

,started_at,PRCP,TMAX
0,2022-11-02 05:54:11.481,NaN,NaN
1,2022-11-02 18:20:15.611,NaN,NaN
2,2022-11-04 18:39:39.873,NaN,NaN
3,2022-11-09 18:02:29.616,NaN,NaN
4,2022-11-12 10:23:11.805,NaN,NaN


In [13]:
citibike_weather["PRCP"].isna().mean()

np.float64(1.0)

In [14]:
type(trips_2022["date"].iloc[0])
type(weather["date"].iloc[0])

str

In [15]:
trips_2022["date"] = pd.to_datetime(trips_2022["started_at"]).dt.normalize()

In [16]:
weather["date"] = pd.to_datetime(weather["date"])

In [17]:
trips_2022["date"].dtype
weather["date"].dtype

dtype('<M8[us]')

In [19]:
# Check both dtypes side-by-side

print("trips_2022 date dtype:", trips_2022["date"].dtype)
print("weather date dtype:", weather["date"].dtype)

trips_2022 date dtype: datetime64[us]
weather date dtype: datetime64[us]


In [20]:
# Inspect actual date values

print(trips_2022["date"].head(5))
print(weather["date"].head(5))

0   2022-11-02
1   2022-11-02
2   2022-11-04
3   2022-11-09
4   2022-11-12
Name: date, dtype: datetime64[us]
0   2022-01-01
1   2022-01-02
2   2022-01-03
3   2022-01-04
4   2022-01-05
Name: date, dtype: datetime64[us]


In [22]:
import numpy as np

In [23]:
# TEST 

# how many unique dates exist in each table?
print("unique trips dates:", trips_2022["date"].nunique())
print("unique weather dates:", weather["date"].nunique())

# do they intersect?
overlap = np.intersect1d(trips_2022["date"].unique(), weather["date"].unique())
print("overlap count:", len(overlap))
print("first 5 overlaps:", overlap[:5])

unique trips dates: 365
unique weather dates: 365
overlap count: 365
first 5 overlaps: ['2022-01-01T00:00:00.000000' '2022-01-02T00:00:00.000000'
 '2022-01-03T00:00:00.000000' '2022-01-04T00:00:00.000000'
 '2022-01-05T00:00:00.000000']


In [24]:
# rebuild a clean key and drop duplicates in weather

# ensure both are normalized (midnight)
trips_2022["date"] = pd.to_datetime(trips_2022["date"]).dt.floor("D")
weather["date"] = pd.to_datetime(weather["date"]).dt.floor("D")

# make sure weather has 1 row per day
weather_clean = weather.drop_duplicates(subset=["date"]).copy()

weather_clean.shape, weather_clean["date"].nunique()

((365, 4), 365)

In [25]:
# Merge again 

citibike_weather_v2 = trips_2022.merge(weather_clean, on="date", how="left")
citibike_weather_v2.shape

(29838166, 17)

In [26]:
# TEST 

print("PRCP NaN rate:", citibike_weather_v2["PRCP"].isna().mean())
print(citibike_weather_v2[["started_at","date","PRCP","TMAX","TMIN"]].head(10))

PRCP NaN rate: 0.0
               started_at       date  PRCP  TMAX  TMIN
0 2022-11-02 05:54:11.481 2022-11-02   0.0  21.7  13.9
1 2022-11-02 18:20:15.611 2022-11-02   0.0  21.7  13.9
2 2022-11-04 18:39:39.873 2022-11-04   0.0  22.2  11.1
3 2022-11-09 18:02:29.616 2022-11-09   0.0  12.2   5.0
4 2022-11-12 10:23:11.805 2022-11-12   2.3  20.6  16.1
5 2022-11-23 08:41:29.302 2022-11-23   0.0  12.8   6.1
6 2022-11-30 00:34:32.008 2022-11-30  10.9  15.0   3.9
7 2022-11-01 18:22:10.732 2022-11-01   4.1  19.4  15.0
8 2022-11-01 19:47:34.369 2022-11-01   4.1  19.4  15.0
9 2022-11-22 15:19:32.105 2022-11-22   0.0  10.6   3.9


In [27]:
# Save 
citibike_weather_v2.to_csv("data/processed/citibike_weather_2022.csv", index=False)

In [28]:
os.listdir("data/processed")

['citibike_trips_weather_merged_2022.csv',
 'citibike_weather_2022.csv',
 'noaa_lga_daily_2022.csv']